In [1]:
from openai import OpenAI
import pandas as pd
from datasets import load_dataset

/Users/donghan/miniconda3/envs/demo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds = load_dataset("/Volumes/lpnp6/MoleculeQA/JSON/All")  # 原始MoleculeQA数据集加载

In [ ]:
chunk_responses = []
for i in range(len(ds["test"])):
    client = OpenAI(
        base_url="https://api.ppinfra.com/v3/openai",
        api_key="sk-xxx",  # <YOUR_API_KEY>
    )
    try:
        data = ds["train"][i]
        question = data["question"]
        bio_seq_type = "smiles"
        question = question.replace("<BIO-SEQ-TYPE>", bio_seq_type)
        question = question.replace("<BIO-SEQ>", data[bio_seq_type])
        user_input = f"Please select the correct answer from the four options below, Please reason step by step, and put your final answer within \\boxed{{}}.\nThe question is: {question}"  # 保持原有提示词格式

        # API请求
        completion = client.chat.completions.create(
            model="deepseek/deepseek-r1/community",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": user_input},
            ],
        )

        # 结果保存
        chunk_responses.append(
            {
                "id": data["id"],
                "cid": data["cid"],
                "answer": data["answer"],
                "response": completion.choices[0].message.content,
            }
        )
        print(completion.choices[0].message.content)
        df = pd.DataFrame(chunk_responses)
        df.to_csv(f"results_{i}.csv", index=False)
    except Exception as e:
        print(f"Error processing index {i}: {e}")

<think>
Okay, let's try to figure this out. I need to determine the absorption characteristics of the molecule based on its SMILES notation. First, I should try to interpret the structure from the SMILES string.

Looking at the SMILES: CN(C)c1ccc(O)c2c1C[C@H]1C[C@H]3[C@H](N(C)C)C(=O)C(C(N)=O)=C(O)[C@@]3(O)C(=O)C1=C2O. Hmm, this looks a bit complex. Let me break it down.

Starting with CN(C): that's a dimethylamine group, maybe part of an aromatic ring substitution. Then there's a benzene ring (c1ccc(O)c2c1), with oxygen (O) substituents. There's a C[C@H]1... which suggests some cyclohexane or similar ring structure with stereochemistry indicated by the @ symbols. The various carbonyl groups (C=O) and amide groups (C(N)=O) are present. Also, several hydroxyl (O) groups and dimethylamino groups (N(C)C).

Wait, the molecule has multiple oxygen and nitrogen-containing functional groups. Let me think about what this structure resembles. Tetracycline antibiotics, perhaps? Tetracyclines have 

In [ ]:
print(chunk_responses)